In [1]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer



,Source System Name,Source System Acronym,Table Name,Table Description
0,Exploration Data System,EDS,SeismicSurveyData_2024,Contains records of seismic surveys conducted ...
1,Exploration Data System,EDS,WellLogAnalysis_2023,Logs from wells including geological strata en...
2,Exploration Data System,EDS,ExplorationDrillingResults,Records of outcomes from exploratory drilling ...
3,Exploration Data System,EDS,GeophysicalStudiesArchive,A compilation of geophysical research studies ...
4,Exploration Data System,EDS,ProspectEvaluationReports,Evaluation reports of potential drilling sites...
5,Exploration Data System,EDS,EnvironmentalImpactPreAssessments,Preliminary assessments of environmental impac...
6,Exploration Data System,EDS,ExploratoryLicenseRegistry,Database of licenses held for exploration incl...
7,Exploration Data System,EDS,DrillingCampaignPerformance,Performance data for different drilling campai...
8,Exploration Data System,EDS,HydrocarbonPotentialZones,Identified zones with potential hydrocarbon pr...
9,Exploration Data System,EDS,FieldDevelopmentPlans,Plans for developing new fields post-discovery...


In [4]:
# Original columns from the dataset
original_columns = data.columns.tolist()
 
# New columns for the generated table structure
generated_columns = ['ColumnNames', 'ColumnAcronyms', 'DataTypes', 'Nullability', 'ColumnDescriptions']
 
# Initialize an empty dataframe to store the results
# Note: We adjust the DataFrame initialization to better accommodate list storage for generated columns
results_df = pd.DataFrame()
 
results_df.head(10)
data.head()

,Source System Name,Source System Acronym,Table Name,Table Description
0,Exploration Data System,EDS,SeismicSurveyData_2024,Contains records of seismic surveys conducted ...
1,Exploration Data System,EDS,WellLogAnalysis_2023,Logs from wells including geological strata en...
2,Exploration Data System,EDS,ExplorationDrillingResults,Records of outcomes from exploratory drilling ...
3,Exploration Data System,EDS,GeophysicalStudiesArchive,A compilation of geophysical research studies ...
4,Exploration Data System,EDS,ProspectEvaluationReports,Evaluation reports of potential drilling sites...


In [9]:
# # THE FIRST TIME YOU RUN THIS, IT MIGHT TAKE A WHILE
# model_path_or_id = "mistralai/Mistral-7B-v0.1"
# tokenizer = AutoTokenizer.from_pretrained(model_path_or_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_path_or_id,
#     torch_dtype=torch.bfloat16,
#     # bnb_4bit_compute_dtype=torch.float16,
#     use_flash_attention_2=True,
#     attn_implementation="flash_attention_2",
#     # load_in_4bit=True,
# ).to("cuda:0")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# THE FIRST TIME YOU RUN THIS, IT MIGHT TAKE A WHILE
model_path_or_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_path_or_id)
model = AutoModelForCausalLM.from_pretrained(
    model_path_or_id,
    torch_dtype=torch.bfloat16,
    bnb_4bit_compute_dtype=torch.float16,
    use_flash_attention_2=True,
    attn_implementation="flash_attention_2",
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
def generate_table_structure(info):
    
    prompt = f"Please generate a table of 5 column names, column acronym, data type, nullability, and column descriptions for the given information: {info}. In CSV format."
    """Convenience function for generating model output"""

#     prompt = f"""
# Task: Generate table rows based on the given column titles.
# Column Titles: ColumnNames, ColumnAcronyms, DataTypes, Nullability, ColumnDescriptions
# Output Format: Provide each row in CSV format.
# Information for row: {info}
#     """
    # Tokenize the input
    input_ids = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True).input_ids.cuda()
   
    # Generate new tokens based on the prompt, up to max_new_tokens
    # Sample aacording to the parameter
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=500,
            do_sample=True,
            top_p=0.9,
            temperature=0.1,
            use_cache=True
        )
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
 
def parse_table_structure(table_text):
    lines = table_text.split('\n')
    table_data = []
    for line in lines:
        fields = line.split(',')
        # Adjust parsing to correctly create lists of lists for each generated row
        if len(fields) >= len(generated_columns):  # Ensure we have the expected number of fields
            table_data.append(fields[:len(generated_columns)])  # Append a list for each row
    return table_data

In [8]:
count = 0

data = data.sample(frac=1).head(40)

# Iterate through each row in the dataset and call the OpenAI API
for index, row in data.iterrows():
    count += 1
    progress = count / len(data)*100
    print(f'{progress:.2f}%')
    generated_table_text = generate_table_structure(row.to_json())
    print(generated_table_text)
    
    generated_table_data = parse_table_structure(generated_table_text)
 
    # Each generated row is added to the results DataFrame
    for generated_row in generated_table_data:
        # Combine original row data with generated data
        combined_row_data = row.tolist() + generated_row
        # Dynamically adjust column names to accommodate both original and generated data
        dynamic_columns = original_columns + [f'Generated_{col}' for col in generated_columns]
        result_row = pd.DataFrame([combined_row_data], columns=dynamic_columns)
        results_df = pd.concat([results_df, result_row], ignore_index=True)
 
# Save the final dataframe to a CSV file
output_file = 'data/generated_table_structures_with_original_data.csv'
results_df.to_csv(output_file, index=False)
 
print(f"Data saved to {output_file}")
 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2.50%


/home/kevinjesse/AccentureTeach/genai-bootcamp-curriculum/genai-bootcamp-curriculum/1-daily-exercises/synthetic-data-exercise/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [3]:
def generate_table_structure(info):
    prompt = f"Please generate a table of 20 column names, column acronym, data type, nullability, and column descriptions for the given information: {info}. In CSV format."
    """Convenience function for generating model output"""
    return prompt

    #     # Tokenize the input
    # input_ids = tokenizer(
    #     prompt,
    #     return_tensors="pt",
    #     truncation=True).input_ids.cuda()
   
    # # Generate new tokens based on the prompt, up to max_new_tokens
    # # Sample aacording to the parameter
    # with torch.inference_mode():
    #     outputs = model.generate(
    #         input_ids,
    #         max_new_tokens=500,
    #         do_sample=True,
    #         top_p=0.9,
    #         temperature=0.1,
    #         use_cache=True
    #     )
    # return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
 
def parse_table_structure(table_text):
    lines = table_text.split('\n')
    table_data = []
    for line in lines:
        fields = line.split(',')
        # Adjust parsing to correctly create lists of lists for each generated row
        if len(fields) >= len(generated_columns):  # Ensure we have the expected number of fields
            table_data.append(fields[:len(generated_columns)])  # Append a list for each row
    return table_data


count = 0
 
create_prompt_list = []
# Iterate through each row in the dataset and call the OpenAI API

prompts = data.apply(lambda x: generate_table_structure(x.to_json()), axis=1)

prompts = list(prompts.values)

from vllm import LLM, SamplingParams
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
model = LLM("mistralai/Mistral-7B-v0.1")
outputs = model.generate(prompts, sampling_params)

tables = []
for output in outputs:
    count += 1

    progress = count / len(data)*100
    print(f'{progress:.2f}%')

    tables.append(parse_table_structure(output.outputs[0].text))
 
# Each generated row is added to the results DataFrame
for generated_row in tables:
    # Combine original row data with generated data
    combined_row_data = row.tolist() + generated_row
    # Dynamically adjust column names to accommodate both original and generated data
    dynamic_columns = original_columns + [f'Generated_{col}' for col in generated_columns]
    result_row = pd.DataFrame([combined_row_data], columns=dynamic_columns)
    results_df = pd.concat([results_df, result_row], ignore_index=True)
 
# Save the final dataframe to a CSV file
output_file = 'data/generated_table_structures_with_original_data.csv'
results_df.to_csv(output_file, index=False)
 
print(f"Data saved to {output_file}")
 

INFO 04-23 19:58:40 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='mistralai/Mistral-7B-v0.1', tokenizer='mistralai/Mistral-7B-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 04-23 19:58:52 selector.py:16] Using FlashAttention backend.


HfHubHTTPError: 504 Server Error: Gateway Time-out for url: https://huggingface.co/api/models/mistralai/Mistral-7B-v0.1